<a href="https://colab.research.google.com/github/lg550055/pe-backend/blob/main/yhooFin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import re
import json
import requests
headers={'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) (KHTML, like Gecko) Chrome/102.0.5005.63'}
ticker = 'WMT'

In [2]:
url = 'https://finance.yahoo.com/quote/'+ ticker + '/analysis?p=' + ticker
page = requests.get(url, headers=headers, timeout=5)
data = json.loads(re.search('root\.App\.main\s*=\s*(.*);', page.text).group(1))["context"]["dispatcher"]["stores"]
# only the data["QuoteSummaryStore"]['earningsTrend']['trend'] object has eps and rev estimates

In [5]:
data['QuoteSummaryStore'].keys()

dict_keys(['recommendationTrend', 'financialsTemplate', 'price', 'earningsHistory', 'indexTrend', 'financialData', 'earningsTrend', 'quoteType', 'sectorTrend', 'summaryDetail', 'symbol', 'upgradeDowngradeHistory', 'pageViews', 'industryTrend'])

In [ ]:
data['QuoteSummaryStore']['financialData'] # good snapshot summary
data['QuoteSummaryStore']['summaryDetail'] # div, and x-div date, highs and lows
data['QuoteSummaryStore']['industryTrend'] # empty
data['QuoteSummaryStore']['sectorTrend'] # empty

In [ ]:
# field = [t for t in data["context"]["dispatcher"]["stores"]["QuoteSummaryStore"]["earningsTrend"]["trend"] if t["period"] == "+5y" ][0]
f = [t for t in data["QuoteSummaryStore"]["earningsTrend"]["trend"] if t["period"] in ['0y','+1y']]
num_analysts = f[0]['earningsEstimate']['numberOfAnalysts']['fmt']
fwd_eps = f[0]['earningsEstimate']['avg']['fmt']
fwd_rev = f[0]['revenueEstimate']['avg']['raw']
fwd_rev_g = f[0]['revenueEstimate']['growth']['raw']
fwd2_eps = f[1]['earningsEstimate']['avg']['fmt']
fwd2_rev = f[1]['revenueEstimate']['avg']['fmt']
fwd2_rev_g = f[1]['revenueEstimate']['growth']['fmt']
print(num_analysts, fwd_eps, fwd_rev, fwd_rev_g, fwd2_eps, fwd2_rev, fwd2_rev_g)
# print("Next 5 Years (per annum) : " + field["growth"]["fmt"])

35 6.41 595703000000 0.04 6.96 616.08B 3.40%


In [69]:
# url for is, bs, cf -but not all items, e.g. q dil shrs or net debt
url = 'https://finance.yahoo.com/quote/'+ ticker + '/financials?p=' + ticker
page = requests.get(url, headers=headers, timeout=5)
data = json.loads(re.search('root\.App\.main\s*=\s*(.*);', page.text).group(1))["context"]["dispatcher"]["stores"]

In [28]:
data["QuoteSummaryStore"].keys()

dict_keys(['financialsTemplate', 'cashflowStatementHistory', 'balanceSheetHistoryQuarterly', 'earnings', 'price', 'incomeStatementHistoryQuarterly', 'incomeStatementHistory', 'balanceSheetHistory', 'cashflowStatementHistoryQuarterly', 'quoteType', 'summaryDetail', 'symbol', 'pageViews'])

In [ ]:
# data["RecommendationStore"]['recommendedSimilarSymbols'] # a few others in the sector
# data["QuoteSummaryStore"]['incomeStatementHistoryQuarterly']['incomeStatementHistory'] # IS items x shr ct
# data["QuoteSummaryStore"]['balanceSheetHistoryQuarterly']['balanceSheetStatements'] # BS items (but not explicit net debt)
# data["QuoteSummaryStore"]['earnings'] # Q and A rev and net income
# data["QuoteSummaryStore"]['summaryDetail'] # price, vol, div, p/e, p/s

In [ ]:
data["QuoteTimeSeriesStore"]['timeSeries'].keys() # annual and trailing IS items

In [87]:
# rev = data["QuoteSummaryStore"]['incomeStatementHistory']['incomeStatementHistory'][0]['totalRevenue']['fmt']
# eop = data["QuoteSummaryStore"]['incomeStatementHistory']['incomeStatementHistory'][0]['endDate']['fmt']
# bseop = data["QuoteSummaryStore"]['balanceSheetHistory']['balanceSheetStatements'][0]['endDate']['fmt']
# ebit = data["QuoteTimeSeriesStore"]['timeSeries']['annualEBIT'][-1]['reportedValue']['raw'] / 1e9
# dshrs = data["QuoteTimeSeriesStore"]['timeSeries']['annualDilutedAverageShares'][-1]['reportedValue']['raw'] / 1e9
# eps = data["QuoteTimeSeriesStore"]['timeSeries']['annualDilutedEPS'][-1]['reportedValue']['fmt'] # for reference in this notebook

eop = data["QuoteTimeSeriesStore"]['timeSeries']['annualTotalRevenue'][-1]['asOfDate']
rev = data["QuoteTimeSeriesStore"]['timeSeries']['annualTotalRevenue'][-1]['reportedValue']['raw'] / 1e9
ebitda = data["QuoteTimeSeriesStore"]['timeSeries']['annualNormalizedEBITDA'][-1]['reportedValue']['raw'] / 1e9
capex = data["QuoteSummaryStore"]['cashflowStatementHistory']['cashflowStatements'][0]['capitalExpenditures']['raw'] / -1e9
cfo = data["QuoteSummaryStore"]['cashflowStatementHistory']['cashflowStatements'][0]['totalCashFromOperatingActivities']['raw'] / 1e9
cfeop = data["QuoteSummaryStore"]['cashflowStatementHistory']['cashflowStatements'][0]['endDate']['fmt']
# print(s)
# print(eps, dshrs, eop, rev, ebit, ebitda, ebitda-capex, cfo, round(ebit/rev*100,1), round(ebitda/rev*100,1))
trev = data["QuoteTimeSeriesStore"]['timeSeries']['trailingTotalRevenue'][-1]['reportedValue']['raw'] / 1e9
treveop = data["QuoteTimeSeriesStore"]['timeSeries']['trailingTotalRevenue'][-1]['asOfDate']
tebitda = data["QuoteTimeSeriesStore"]['timeSeries']['trailingNormalizedEBITDA'][-1]['reportedValue']['raw'] / 1e9
print(ticker)
print('prev yr')
print(eop,rev,ebitda, capex, cfo, cfeop)
print('trailing')
print(treveop,trev,tebitda)

WMT
prev yr
2022-01-31 572.754 33.758 13.106 24.181 2022-01-31
trailing
2022-04-30 576.013 32.723


In [89]:
# url for trailing CF items
url = 'https://finance.yahoo.com/quote/'+ ticker + '/cash-flow?p=' + ticker
page = requests.get(url, headers=headers, timeout=5)
data = json.loads(re.search('root\.App\.main\s*=\s*(.*);', page.text).group(1))["context"]["dispatcher"]["stores"]

In [ ]:
data['QuoteTimeSeriesStore']['timeSeries'].keys()

In [97]:
# data['QuoteTimeSeriesStore']['timeSeries']['annualCapitalExpenditure'] # list w 4 obj
tcapex = data['QuoteTimeSeriesStore']['timeSeries']['trailingCapitalExpenditure'][0]['reportedValue']['raw'] / 1e9 # list w 1 obj
tcfo = data['QuoteTimeSeriesStore']['timeSeries']['trailingOperatingCashFlow'][0]['reportedValue']['raw'] / -1e9 # list w 1 obj
print('trailing', tcapex, tcfo)

trailing -14.431 -17.565


In [41]:
# url for netDebt
url = 'https://finance.yahoo.com/quote/'+ ticker + '/balance-sheet?p=' + ticker
page = requests.get(url, headers=headers, timeout=5)
data = json.loads(re.search('root\.App\.main\s*=\s*(.*);', page.text).group(1))["context"]["dispatcher"]["stores"]

In [ ]:
data["QuoteTimeSeriesStore"]['timeSeries']['annualNetDebt'][-1] # timeSeries is only object with data, has annual BS items

{'asOfDate': '2022-01-31',
 'currencyCode': 'USD',
 'dataId': 23531,
 'periodType': '12M',
 'reportedValue': {'fmt': '23.32B', 'raw': 23317000000}}

In [47]:
url = 'https://finance.yahoo.com/quote/'+ ticker + '/profile?p=' + ticker
page = requests.get(url, headers=headers, timeout=5)
data = json.loads(re.search('root\.App\.main\s*=\s*(.*);', page.text).group(1))["context"]["dispatcher"]["stores"]
data.keys()

dict_keys(['PageStore', 'MRTStore', 'RouteStore', 'I13nStore', 'PageTransitionStore', 'VideoPlayerStore', 'QuoteAutoCompleteStore', 'NavrailStore', 'StreamDataStore', 'QuoteSummaryStore', 'FinanceConfigStore', 'LangStore', 'BeaconStore', 'AdStore', 'VideoStore', 'ComponentConfigStore', 'CrumbStore', 'CompositeStore', 'MonalixaStore', 'StreamStore', 'UserStore', 'ProfileStore', 'QuotePageStore', 'MarketTimeStore', 'MarketSummaryStore', 'NavServiceStore', 'RecommendationStore', 'UHAccountSwitchStore', 'RmpStore', 'MobileHeaderStore'])

In [48]:
data['QuoteSummaryStore'].keys()

dict_keys(['financialsTemplate', 'price', 'secFilings', 'quoteType', 'calendarEvents', 'summaryDetail', 'symbol', 'assetProfile', 'pageViews'])

In [52]:
# data['QuoteSummaryStore']['secFilings'] # most recent sec filings
data['QuoteSummaryStore']['assetProfile'].keys()

dict_keys(['zip', 'sector', 'fullTimeEmployees', 'compensationRisk', 'auditRisk', 'longBusinessSummary', 'city', 'phone', 'state', 'shareHolderRightsRisk', 'compensationAsOfEpochDate', 'governanceEpochDate', 'boardRisk', 'country', 'companyOfficers', 'website', 'maxAge', 'overallRisk', 'address1', 'industry'])

In [54]:
data['QuoteSummaryStore']['assetProfile']['sector'] # invesment, e.g. WMT = consumer defensive
data['QuoteSummaryStore']['assetProfile']['industry'] # E.g. WMT = Discount Stores

'Discount Stores'

In [55]:
url = 'https://finance.yahoo.com/quote/'+ ticker + '/key-statistics?p=' + ticker
page = requests.get(url, headers=headers, timeout=5)
data = json.loads(re.search('root\.App\.main\s*=\s*(.*);', page.text).group(1))["context"]["dispatcher"]["stores"]
data.keys()

dict_keys(['PageStore', 'MRTStore', 'RouteStore', 'I13nStore', 'PageTransitionStore', 'VideoPlayerStore', 'QuoteAutoCompleteStore', 'NavrailStore', 'StreamDataStore', 'QuoteSummaryStore', 'FinanceConfigStore', 'LangStore', 'BeaconStore', 'AdStore', 'VideoStore', 'ComponentConfigStore', 'CrumbStore', 'CompositeStore', 'MonalixaStore', 'StreamStore', 'UserStore', 'ProfileStore', 'QuotePageStore', 'QuoteTimeSeriesStore', 'MarketTimeStore', 'MarketSummaryStore', 'RecommendationStore', 'RmpStore', 'NavServiceStore', 'UHAccountSwitchStore', 'MobileHeaderStore'])

In [60]:
data['QuoteTimeSeriesStore'].keys()
data['QuoteTimeSeriesStore']['timeSeries'].keys()

dict_keys(['quarterlyEnterprisesValueEBITDARatio', 'trailingMarketCap', 'trailingPegRatio', 'quarterlyPeRatio', 'quarterlyMarketCap', 'trailingEnterprisesValueEBITDARatio', 'quarterlyPegRatio', 'quarterlyPsRatio', 'trailingEnterpriseValue', 'quarterlyEnterpriseValue', 'trailingForwardPeRatio', 'trailingPeRatio', 'quarterlyEnterprisesValueRevenueRatio', 'quarterlyForwardPeRatio', 'trailingPsRatio', 'trailingPbRatio', 'quarterlyPbRatio', 'trailingEnterprisesValueRevenueRatio', 'timestamp'])

In [61]:
data['QuoteSummaryStore'].keys()

dict_keys(['defaultKeyStatistics', 'financialsTemplate', 'price', 'financialData', 'quoteType', 'calendarEvents', 'summaryDetail', 'symbol', 'pageViews'])

In [62]:
data['QuoteSummaryStore']['defaultKeyStatistics'].keys()

dict_keys(['annualHoldingsTurnover', 'enterpriseToRevenue', 'beta3Year', 'profitMargins', 'enterpriseToEbitda', '52WeekChange', 'morningStarRiskRating', 'forwardEps', 'revenueQuarterlyGrowth', 'sharesOutstanding', 'fundInceptionDate', 'annualReportExpenseRatio', 'totalAssets', 'bookValue', 'sharesShort', 'sharesPercentSharesOut', 'fundFamily', 'lastFiscalYearEnd', 'heldPercentInstitutions', 'netIncomeToCommon', 'trailingEps', 'lastDividendValue', 'SandP52WeekChange', 'priceToBook', 'heldPercentInsiders', 'nextFiscalYearEnd', 'yield', 'mostRecentQuarter', 'shortRatio', 'sharesShortPreviousMonthDate', 'floatShares', 'beta', 'enterpriseValue', 'priceHint', 'threeYearAverageReturn', 'lastSplitDate', 'lastSplitFactor', 'legalType', 'lastDividendDate', 'morningStarOverallRating', 'earningsQuarterlyGrowth', 'priceToSalesTrailing12Months', 'dateShortInterest', 'pegRatio', 'ytdReturn', 'forwardPE', 'maxAge', 'lastCapGain', 'shortPercentOfFloat', 'sharesShortPriorMonth', 'impliedSharesOutstandin

In [64]:
data['QuoteSummaryStore']['defaultKeyStatistics']['sharesOutstanding']['raw'] / 1e9

{'fmt': '2.74B', 'longFmt': '2,741,149,952', 'raw': 2741149952}

In [ ]:
data['QuoteSummaryStore']['financialData'] # price, fins snapshot summary

## Selenium

In [ ]:
!pip install selenium
!apt-get update 
!apt install chromium-chromedriver

from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

In [ ]:
d = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
ss = input('Stock symbol?')
d.get(f'https://finance.yahoo.com/quote/{ss}/analysis?p={ss}')
main = d.find_element_by_id('Main')
tds = main.find_elements_by_tag_name('td')
if tds:
  data = [e.text for e in tds]
  print(data[0],data[4],data[5],data[8],data[9],data[30],data[33],data[34],data[50],data[53],data[54])
else:
  print('no data')
d.quit()